In [1]:
import pandas as pd
import numpy as np
import requests
import json
import os
from pyspark.sql import SparkSession
import pyspark.pandas as ps
from pyspark.sql.functions import udf, col, explode
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType, ArrayType, DateType
from datetime import datetime, timedelta

os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

In [5]:
spark = SparkSession.builder.master('local[*]').appName('CNEOS_Data_Extractor').getOrCreate()
spark

In [6]:
date = datetime.today().date()
date_min = date + timedelta(days=59)
date_max = date + timedelta(days=60)
print(date_min, date_max)

2023-03-13 2023-03-14


In [7]:
def makeAPICall(url, parameters):
    try:
        response = requests.get(url, params=parameters)
    except Exception as e:
        return e

    if response.status_code == 200 and response != None:
        return response.json()
    return None

In [8]:
url = "https://ssd-api.jpl.nasa.gov/cad.api"
parameters = {
    "date-min": str(datetime.today().date()),
    # "date-min": "1900-01-04",
    "date-max": str(date_max),
    "dist-max": "0.01",
    'fullname': "true",
    # 'dist-max': "0.1",
    'diameter': "true"
}
# response = requests.get(url, parameters)
# data = response.json()

# response = makeAPICall(url, parameters)

In [9]:
# response['count']
# data['count']

In [10]:
columns = [
    'Designation',
    'Orbit Id',
    'Time of Close approach',
    'Close-Approach Date',
    'Nominal Approch distance (au)',
    'Min Close-Approach Distance (au)',
    'Max Close-Approach Distance (au)',
    'V Reletive (Km/s)',
    'V Infinite (Km/s)',
    'Close-Approach Uncertain Time',
    'Absolute Magnitude (mag)',
    'Diameter (Km)',
    'Diameter-Sigma (Km)',
    'Object'
]


In [11]:
schema = StructType([
    StructField("Signature", StringType(), True),
    StructField("Count", StringType(), True),
    StructField("Fields", ArrayType(
        StructType([
            StructField("Designation", StringType(), True),
            StructField("Orbit_Id", StringType(), True),
            StructField("Time of Close approach", StringType(), True),
            StructField("Close-Approach Date", StringType(), True),
            StructField("Nominal Approch distance (au)", StringType(), True),
            StructField("Min Close-Approach Distance (au)",
                        StringType(), True),
            StructField("Max Close-Approach Distance (au)",
                        StringType(), True),
            StructField("V Reletive (Km/s)", StringType(), True),
            StructField("V Infinite (Km/s)", StringType(), True),
            StructField("Close-Approach Uncertain Time", StringType(), True),
            StructField("Absolute Magnitude (mag)", StringType(), True),
            StructField("Diameter (Km)", StringType(), True),
            StructField("Diameter-Sigma (Km)", StringType(), True),
            StructField("Designation", StringType(), True),
        ])
    ), True),
    StructField("Data", ArrayType(
        StructType([
            StructField("Designation", StringType(), True),
            StructField("Orbit_Id", StringType(), True),
            StructField("Time of Close approach", DoubleType(), True),
            StructField("Close-Approach Date", DateType(), True),
            StructField("Nominal Approch distance (au)", DoubleType(), True),
            StructField("Min Close-Approach Distance (au)",
                        DoubleType(), True),
            StructField("Max Close-Approach Distance (au)",
                        DoubleType(), True),
            StructField("V Reletive (Km/s)", DoubleType(), True),
            StructField("V Infinite (Km/s)", DoubleType(), True),
            StructField("Close-Approach Uncertain Time", StringType(), True),
            StructField("Absolute Magnitude (mag)", DoubleType(), True),
            StructField("Diameter (Km)", DoubleType(), True),
            StructField("Diameter-Sigma (Km)", DoubleType(), True),
            StructField("Designation", StringType(), True),
        ])
    ), True)
])
schema


StructType([StructField('Signature', StringType(), True), StructField('Count', StringType(), True), StructField('Fields', ArrayType(StructType([StructField('Designation', StringType(), True), StructField('Orbit_Id', StringType(), True), StructField('Time of Close approach', StringType(), True), StructField('Close-Approach Date', StringType(), True), StructField('Nominal Approch distance (au)', StringType(), True), StructField('Min Close-Approach Distance (au)', StringType(), True), StructField('Max Close-Approach Distance (au)', StringType(), True), StructField('V Reletive (Km/s)', StringType(), True), StructField('V Infinite (Km/s)', StringType(), True), StructField('Close-Approach Uncertain Time', StringType(), True), StructField('Absolute Magnitude (mag)', StringType(), True), StructField('Diameter (Km)', StringType(), True), StructField('Diameter-Sigma (Km)', StringType(), True), StructField('Designation', StringType(), True)]), True), True), StructField('Data', ArrayType(StructTyp

In [12]:
response = makeAPICall(url, parameters)
#  = response.json()
response['data']
# udf_df = udf(response, schema)

[['2023 AH',
  '1',
  '2459959.899135528',
  '2023-Jan-15 09:35',
  '0.00473299845260497',
  '0.00471915186629017',
  '0.00474684112006944',
  '4.78415040125375',
  '4.66499507650631',
  '00:03',
  '28.552',
  None,
  None,
  '       (2023 AH)'],
 ['2020 DG4',
  '7',
  '2459993.445587536',
  '2023-Feb-17 22:42',
  '0.00369244188194289',
  '0.00297819606291202',
  '0.0276770682501224',
  '6.92028867583675',
  '6.81521739682228',
  '4_08:56',
  '28.2',
  None,
  None,
  '       (2020 DG4)']]

In [13]:
def formattingData(data):
    formattedData = []
    for row in data:
        temp = {}
        for j in range(len(row)):
            temp[columns[j]] = row[j]
        formattedData.append(temp)
    return formattedData


In [14]:
formattingData(response['data'])

[{'Designation': '2023 AH',
  'Orbit Id': '1',
  'Time of Close approach': '2459959.899135528',
  'Close-Approach Date': '2023-Jan-15 09:35',
  'Nominal Approch distance (au)': '0.00473299845260497',
  'Min Close-Approach Distance (au)': '0.00471915186629017',
  'Max Close-Approach Distance (au)': '0.00474684112006944',
  'V Reletive (Km/s)': '4.78415040125375',
  'V Infinite (Km/s)': '4.66499507650631',
  'Close-Approach Uncertain Time': '00:03',
  'Absolute Magnitude (mag)': '28.552',
  'Diameter (Km)': None,
  'Diameter-Sigma (Km)': None,
  'Object': '       (2023 AH)'},
 {'Designation': '2020 DG4',
  'Orbit Id': '7',
  'Time of Close approach': '2459993.445587536',
  'Close-Approach Date': '2023-Feb-17 22:42',
  'Nominal Approch distance (au)': '0.00369244188194289',
  'Min Close-Approach Distance (au)': '0.00297819606291202',
  'Max Close-Approach Distance (au)': '0.0276770682501224',
  'V Reletive (Km/s)': '6.92028867583675',
  'V Infinite (Km/s)': '6.81521739682228',
  'Close-Ap

In [15]:
data = formattingData(response['data'])

In [16]:
df = ps.DataFrame(data)
df.head()

c:\Users\vaibh\anaconda3\envs\azure\lib\site-packages\pyspark\pandas\internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
c:\Users\vaibh\anaconda3\envs\azure\lib\site-packages\pyspark\sql\pandas\conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


Py4JJavaError: An error occurred while calling o170.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (VAibhAv executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:189)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:164)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:365)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:329)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "c:\Users\vaibh\anaconda3\envs\azure\lib\site-packages\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "c:\Users\vaibh\anaconda3\envs\azure\lib\site-packages\py4j\clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "c:\Users\vaibh\anaconda3\envs\azure\lib\socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [60]:
df = spark.createDataFrame(data=data, schema=schema)

In [61]:
df.show(1)

Py4JJavaError: An error occurred while calling o84.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 24) (VAibhAv executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:189)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:164)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:365)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:329)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.s

In [50]:
udf_df = udf(formattingData, schema)

In [54]:
# options = { 'url' : url, 'method' : 'GET', 'readTimeout' : '10000', 'connectionTimeout' : '2000', 'partitions' : '10'}

# tdf = spark.read.format("org.apache.dsext.spark.datasource.rest.RestDataSource").options(**options).load()

In [ ]:
tdf = spark.read.json()

In [58]:
sc = spark.sparkContext
json_content1 = "{'json_col1': 'hello', 'json_col2': 32}"
json_content2 = "{'json_col1': 'hello', 'json_col2': 'world'}"

json_list = []
json_list.append(json_content1)
json_list.append(json_content2)

sdf = spark.read.json(sc.parallelize(json_list))


Py4JJavaError: An error occurred while calling o55.json.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 10 in stage 1.0 failed 1 times, most recent failure: Lost task 10.0 in stage 1.0 (TID 22) (VAibhAv executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:189)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:164)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:365)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:329)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala